<a href="https://colab.research.google.com/github/bryanbayup/chatbot_project/blob/main/multi_turn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gensim keras-tuner imbalanced-learn Sastrawi sentencepiece seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 8.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=1a141dab1a82d558e3bbef0c6b31839050a2f44ecb48b37467f7eca0b401e00c
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
# Download FastText
!wget -O id.tar.gz "https://www.dropbox.com/scl/fi/sju4o3keikox69euw51vy/id.tar.gz?rlkey=5jr3ijtbdwfahq7xcgig28qvy&e=1&st=gntzkzeo&dl=1"
!tar -xzf id.tar.gz

--2024-12-07 03:00:10--  https://www.dropbox.com/scl/fi/sju4o3keikox69euw51vy/id.tar.gz?rlkey=5jr3ijtbdwfahq7xcgig28qvy&e=1&st=gntzkzeo&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3d0cf0333cda9c57fd3ece5f81.dl.dropboxusercontent.com/cd/0/inline/Cfx5j_BSCLKaecs7k71PY4OVK3b1eGOZsKcligQZ5uuypvaF9BKuK4FRYgCVIBgwlvXFvLe_I2uEyK0w6X4NSvZhRHwVaZA6Nyc9Afanggd5xLcgeD2JVpPzlR5wyEPENnI/file?dl=1# [following]
--2024-12-07 03:00:11--  https://uc3d0cf0333cda9c57fd3ece5f81.dl.dropboxusercontent.com/cd/0/inline/Cfx5j_BSCLKaecs7k71PY4OVK3b1eGOZsKcligQZ5uuypvaF9BKuK4FRYgCVIBgwlvXFvLe_I2uEyK0w6X4NSvZhRHwVaZA6Nyc9Afanggd5xLcgeD2JVpPzlR5wyEPENnI/file?dl=1
Resolving uc3d0cf0333cda9c57fd3ece5f81.dl.dropboxusercontent.com (uc3d0cf0333cda9c57fd3ece5f81.dl.dropboxusercontent.com)... 162.125.5.15, 2620:100:601d:15::a2

In [4]:
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import (Dense, Input, Dropout, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D,
                                     TimeDistributed, Embedding, GlobalAveragePooling1D, Layer, Lambda)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import re
import pickle
import os
import random
import nltk
import gensim
from gensim.models import KeyedVectors
from keras_tuner import HyperModel, RandomSearch
from tensorflow.keras.callbacks import EarlyStopping
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from seqeval.metrics import classification_report as seq_classification_report
nltk.download('stopwords')

# --------------------------------------
# Load FastText
# --------------------------------------
try:
    fasttext_model = KeyedVectors.load_word2vec_format('id.vec', binary=False)
    print("FastText 'id.vec' berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat 'id.vec': {e}")
    raise ValueError("Gagal memuat FastText.")

# --------------------------------------
# Load Data
# Pastikan 'data2.json' adalah file yang berisi dataset terakhir yang Anda berikan.
# --------------------------------------
with open('data2.json', 'r', encoding='utf-8') as f:
    conversations = json.load(f)

# --------------------------------------
# Ekstrak Data
# --------------------------------------
def char_offset_to_token_labels(utterance, entities, tokenizer=lambda x: x.split()):
    tokens = tokenizer(utterance)
    labels = ["O"] * len(tokens)
    char_pos = 0
    token_ranges = []
    for t in tokens:
        start_pos = char_pos
        end_pos = start_pos + len(t)
        token_ranges.append((start_pos, end_pos))
        char_pos = end_pos + 1
    for ent in entities:
        ent_start = ent['start']
        ent_end = ent['end']
        ent_type = ent['entity'].upper()
        ent_token_positions = []
        for i, (ts, te) in enumerate(token_ranges):
            if not (te <= ent_start or ts >= ent_end):
                ent_token_positions.append(i)
        if len(ent_token_positions) > 0:
            labels[ent_token_positions[0]] = "B-" + ent_type
            for p in ent_token_positions[1:]:
                labels[p] = "I-" + ent_type
    return tokens, labels

user_utterances = []
intents = []
entity_labels = []

for conv in conversations:
    for turn in conv["turns"]:
        if turn["speaker"] == "user":
            utt = turn["utterance"]
            ents = turn.get("entities", [])
            intent = turn.get("intent", "None")
            tokens, ner_tags = char_offset_to_token_labels(utt, ents)
            user_utterances.append(tokens)
            intents.append(intent)
            entity_labels.append(ner_tags)

# --------------------------------------
# Preprocessing Text
# --------------------------------------
with open('stopword_list_tala.txt', 'r', encoding='utf-8') as f:
    stop_words = f.read().splitlines()
stop_words = set(word.strip().lower() for word in stop_words)

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def preprocess_text(text):
    text = clean_text(text)
    tokens = text.split()
    tokens = [w for w in tokens if w not in stop_words]
    tokens = [stemmer.stem(w) for w in tokens]
    return ' '.join(tokens)

utterances_joined = [' '.join(utt) for utt in user_utterances]
utterances_clean = [preprocess_text(u) for u in utterances_joined]

df_data = pd.DataFrame({
    'utterances': utterances_joined,
    'intent': intents,
    'entities': entity_labels,
    'utterances_clean': utterances_clean
})

label_encoder = LabelEncoder()
df_data['intent_label'] = label_encoder.fit_transform(df_data['intent'])

# Balance data
ros = RandomOverSampler(random_state=42)
X = df_data.index.values.reshape(-1, 1)
y = df_data['intent_label']
X_ros, y_ros = ros.fit_resample(X, y)
df_balanced = df_data.loc[X_ros.flatten()].reset_index(drop=True)
df_balanced['intent_label'] = y_ros
df_balanced['intent'] = label_encoder.inverse_transform(df_balanced['intent_label'])

# --------------------------------------
# Split Data for Intent Classification
# --------------------------------------
texts = df_balanced['utterances_clean'].tolist()
labels = df_balanced['intent_label'].tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='')
tokenizer.fit_on_texts(train_texts)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)
max_seq_length = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in val_sequences))
train_padded = pad_sequences(train_sequences, maxlen=max_seq_length, padding='post')
val_padded = pad_sequences(val_sequences, maxlen=max_seq_length, padding='post')

num_classes = len(label_encoder.classes_)
train_labels_cat = to_categorical(train_labels, num_classes=num_classes)
val_labels_cat = to_categorical(val_labels, num_classes=num_classes)

embedding_dim = 300  # Gunakan embedding dim yang sama seperti fasttext
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, idx in word_index.items():
    if word in fasttext_model:
        embedding_matrix[idx] = fasttext_model[word]
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

# --------------------------------------
# NER Data
# --------------------------------------
all_labels = set()
for tags in df_balanced['entities']:
    for t in tags:
        if t != 'O':
            all_labels.add(t)
all_labels.add('O')
all_labels = sorted(list(all_labels))
ner_label_encoder = {label: idx for idx, label in enumerate(all_labels)}
ner_label_decoder = {idx: label for label, idx in ner_label_encoder.items()}

def encode_tags(tags, max_len):
    tag_ids = [ner_label_encoder[t] for t in tags]
    tag_ids = tag_ids[:max_len] + [ner_label_encoder['O']]*(max_len - len(tag_ids))
    return tag_ids

def text_to_sequence(text):
    seq = tokenizer.texts_to_sequences([text])
    return seq[0]

X_ner = []
Y_ner = []
for i, row in df_balanced.iterrows():
    seq = text_to_sequence(row['utterances_clean'])
    seq_padded = seq[:max_seq_length] + [0]*(max_seq_length - len(seq))
    X_ner.append(seq_padded)
    tag_ids = encode_tags(row['entities'], max_seq_length)
    Y_ner.append(tag_ids)

X_ner = np.array(X_ner)
Y_ner = np.array(Y_ner)
Y_ner = to_categorical(Y_ner, num_classes=len(ner_label_encoder))

train_texts_ner, val_texts_ner, train_labels_ner, val_labels_ner = train_test_split(
    X_ner,
    Y_ner,
    test_size=0.2,
    random_state=42
)

# --------------------------------------
# Attention Layer
# --------------------------------------
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='att_weight', shape=(input_shape[-1], 1),
                                 initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(name='att_bias', shape=(1,),
                                 initializer='zeros', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = tf.squeeze(tf.tensordot(x, self.W, axes=1), axis=-1) + self.b
        alpha = tf.nn.softmax(e)
        alpha = tf.expand_dims(alpha, axis=-1)
        context = x * alpha
        return tf.reduce_sum(context, axis=1)  # shape: (batch, features)

# --------------------------------------
# Model Intent dengan BiLSTM + Attention
# --------------------------------------
def build_intent_model(embedding_matrix, max_seq_length, num_classes, l2_reg=1e-3):
    inputs = Input(shape=(max_seq_length,), dtype='int32')
    emb = Embedding(input_dim=embedding_matrix.shape[0],
                    output_dim=embedding_matrix.shape[1],
                    weights=[embedding_matrix],
                    input_length=max_seq_length,
                    trainable=True, mask_zero=True)(inputs)
    x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))(emb)
    att = AttentionLayer()(x)
    dense = Dense(128, activation='relu', kernel_regularizer=l2(l2_reg))(att)
    dropout = Dropout(0.5)(dense)
    outputs = Dense(num_classes, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model_intent = build_intent_model(embedding_matrix, max_seq_length, num_classes)
early_intent = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_intent.fit(
    train_padded, train_labels_cat,
    validation_data=(val_padded, val_labels_cat),
    epochs=30, batch_size=16,
    callbacks=[early_intent]
)

# --------------------------------------
# Model NER dengan BiLSTM + Attention
# --------------------------------------
def build_ner_model(embedding_matrix, max_seq_length, num_entities, l2_reg=1e-3):
    inputs = Input(shape=(max_seq_length,))
    emb = Embedding(input_dim=embedding_matrix.shape[0],
                    output_dim=embedding_matrix.shape[1],
                    weights=[embedding_matrix],
                    trainable=True)(inputs) # mask_zero dihapus
    x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))(emb)
    x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))(x)

    # Tetap gunakan TimeDistributed Dense untuk output NER
    td = TimeDistributed(Dense(num_entities, activation='softmax'))(x)
    model = Model(inputs, td)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model_ner = build_ner_model(embedding_matrix, max_seq_length, len(ner_label_encoder))
early_ner = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_ner.fit(
    train_texts_ner, train_labels_ner,
    validation_data=(val_texts_ner, val_labels_ner),
    epochs=30, batch_size=16,
    callbacks=[early_ner]
)

# Evaluate
loss_intent, acc_intent = model_intent.evaluate(val_padded, val_labels_cat)
print("Akurasi Intent:", acc_intent)

loss_ner, acc_ner = model_ner.evaluate(val_texts_ner, val_labels_ner)
print("Akurasi NER:", acc_ner)

# --------------------------------------
# Simpan Model
# --------------------------------------
os.makedirs('models', exist_ok=True)
os.makedirs('encoders', exist_ok=True)
os.makedirs('data', exist_ok=True)

model_intent.save('models/model_intent.keras')
model_ner.save('models/model_ner.keras')

with open('encoders/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('encoders/label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('encoders/ner_label_encoder.pickle', 'wb') as handle:
    pickle.dump(ner_label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_utterances = df_balanced[['utterances', 'intent', 'utterances_clean']].reset_index(drop=True)
vectorizer = TfidfVectorizer()
vectorizer.fit(df_utterances['utterances_clean'])
with open('data/vectorizer.pickle', 'wb') as handle:
    pickle.dump(vectorizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

intent_animal_mapping = {
    "Melaporkan Hewan Terlantar": ["kucing", "anjing"],
    "Mendiagnosis Gejala": ["kucing", "anjing"],
    "Rekomendasi Penanganan Awal": ["kucing", "anjing"],
    "Konfirmasi Laporan": ["kucing", "anjing"],
    "Tindak Lanjut Laporan": ["kucing", "anjing"],
    "Rekomendasi Tindakan": ["kucing", "anjing"]
}

def predict_intent(text):
    text_clean = preprocess_text(text)
    seq = tokenizer.texts_to_sequences([text_clean])
    padded = pad_sequences(seq, maxlen=max_seq_length, padding='post')
    pred = model_intent.predict(padded)
    predicted_label = np.argmax(pred, axis=1)[0]
    return label_encoder.inverse_transform([predicted_label])[0]

def predict_entities(text):
    text_clean = preprocess_text(text)
    seq = tokenizer.texts_to_sequences([text_clean])
    padded = pad_sequences(seq, maxlen=max_seq_length, padding='post')
    pred = model_ner.predict(padded)
    pred_labels = np.argmax(pred[0], axis=-1)
    tokens = tokenizer.sequences_to_texts(seq)[0].split()
    entities = []
    for idx, label_id in enumerate(pred_labels[:len(tokens)]):
        label = ner_label_decoder[label_id]
        if label != 'O':
            entities.append({'entity': label.split('-')[1].lower(), 'value': tokens[idx]})
    return entities

def adjust_intent(intent, entities):
    predicted_animals = intent_animal_mapping.get(intent, None)
    entity_animals = [ent['value'].lower() for ent in entities if ent['entity'] == 'animal']
    if entity_animals and predicted_animals:
        user_animal = entity_animals[0]
        if user_animal not in predicted_animals:
            for i_name, animals in intent_animal_mapping.items():
                if user_animal in animals:
                    intent = i_name
                    break
            else:
                intent = None
    return intent

def get_default_response():
    default_responses = [
        "Maaf, saya belum bisa menjawab pertanyaan Anda.",
        "Mohon diperjelas, saya belum mengerti konteksnya.",
        "Silakan berikan informasi lebih detail.",
        "Maaf, saya hanya diprogram untuk menjawab mengenai kucing dan anjing.",
        "Saya sarankan konsultasi langsung ke dokter hewan."
    ]
    return random.choice(default_responses)

def get_response(user_input, intent=None, entities=None):
    # Untuk kesederhanaan, kita berikan respon tergantung intent:
    # Jika tidak puas dengan ini, Anda bisa membuat mapping intent ke response yang lebih baik.
    if intent == "Mendiagnosis Gejala":
        return "Berdasarkan gejalanya, kemungkinan ada masalah kesehatan. Saya sarankan segera periksakan ke dokter hewan."
    elif intent == "Rekomendasi Penanganan Awal":
        return "Cobalah langkah penanganan awal seperti menjaga kebersihan, memberikan makanan ringan, dan konsultasikan ke dokter hewan jika berlanjut."
    else:
        return get_default_response()

dialog_history = []
MAX_HISTORY = 2

def predict_intent_multi_turn(dialog_history, current_input):
    history_context = dialog_history[-MAX_HISTORY:] if len(dialog_history) > MAX_HISTORY else dialog_history
    combined_input = " ".join(history_context + [current_input])
    return predict_intent(combined_input)

def predict_entities_multi_turn(dialog_history, current_input):
    history_context = dialog_history[-MAX_HISTORY:] if len(dialog_history) > MAX_HISTORY else dialog_history
    combined_input = " ".join(history_context + [current_input])
    return predict_entities(combined_input)

def chatbot_response_multi_turn(user_input):
    dialog_history.append(user_input)
    intent = predict_intent_multi_turn(dialog_history, user_input)
    entities = predict_entities_multi_turn(dialog_history, user_input)
    adjusted = adjust_intent(intent, entities)
    if adjusted is None:
        response = get_default_response()
    else:
        response = get_response(user_input, adjusted, entities)
        if not response:
            response = get_default_response()
    return response

# --------------------------------------
# TESTING DENGAN PERTANYAAN YANG DIMINTA
# Gunakan data "conv_043", "conv_044", "conv_045" yang telah Anda berikan.
# --------------------------------------

print("=== Test Model dengan Pertanyaan dari conv_043 ===")
dialog_history.clear()
test_turns_043 = [
    "Saya melihat seekor kucing sakit di depan toko.",
    "Kucingnya dalam keadaan sehat"
]

for t in test_turns_043:
    print("User:", t)
    resp = chatbot_response_multi_turn(t)
    print("Chatbot:", resp, "\n")

print("=== Test Model dengan Pertanyaan dari conv_044 ===")
dialog_history.clear()
test_turns_044 = [
    "Anjing saya tidak nafsu makan dan sering diare. Apa yang terjadi?",
    "Tidak, hanya diare saja."
]

for t in test_turns_044:
    print("User:", t)
    resp = chatbot_response_multi_turn(t)
    print("Chatbot:", resp, "\n")

print("=== Test Model dengan Pertanyaan dari conv_045 ===")
dialog_history.clear()
test_turns_045 = [
    "Kucing saya sering mengeluarkan air mata dan matanya terlihat merah.",
    "Ya, ada renovasi rumah dengan banyak debu."
]

for t in test_turns_045:
    print("User:", t)
    resp = chatbot_response_multi_turn(t)
    print("Chatbot:", resp, "\n")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


FastText 'id.vec' berhasil dimuat.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'attention_layer' (of type AttentionLayer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Epoch 1/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 83ms/step - accuracy: 0.4038 - loss: 1.7261 - val_accuracy: 0.6286 - val_loss: 1.3319
Epoch 2/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.6350 - loss: 1.2036 - val_accuracy: 0.8571 - val_loss: 0.7381
Epoch 3/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.8654 - loss: 0.6800 - val_accuracy: 0.8429 - val_loss: 0.5018
Epoch 4/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.8830 - loss: 0.4696 - val_accuracy: 0.9000 - val_loss: 0.3815
Epoch 5/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.9712 - loss: 0.2749 - val_accuracy: 0.8857 - val_loss: 0.3459
Epoch 6/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.9766 - loss: 0.2220 - val_accuracy: 0.9143 - val_loss: 0.3347
Epoch 7/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.9872 - loss: 0.1826 - val_accuracy: 0.9143 - val_loss: 0.3949
Epoch 8/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.9961 - loss: 0.1546 - val_accuracy: 0.9143 - v

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'attention_layer' (of type AttentionLayer) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 747ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Chatbot: Saya sarankan konsultasi langsung ke dokter hewan. 

User: Kucingnya dalam keadaan sehat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Chatbot: Mohon diperjelas, saya belum mengerti konteksnya. 

=== Test Model dengan Pertanyaan dari conv_044 ===
User: Anjing saya tidak nafsu makan dan sering diare. Apa yang terjadi?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Chatbot: Berdasarkan gejalanya, kemungkinan ada masalah kesehatan. Saya sarankan segera periksakan ke dokter hewan. 

User: Tidak, hanya diare saja.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Chatbot: Saya sarankan konsultasi langsung ke dokter hewan. 

=== Test Model dengan Pertanyaan dari conv_045 ===
User: Kucing saya sering mengeluarkan air mata dan matanya terlihat merah.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Ch

In [6]:
import json
import pandas as pd
import re
import random
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')

# Pastikan file data sudah diupload, misalnya data2.json
with open('data2.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

with open('stopword_list_tala.txt', 'r', encoding='utf-8') as f:
    stop_words = f.read().splitlines()
stop_words = set(w.strip().lower() for w in stop_words)

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def preprocess_text(text):
    text = clean_text(text)
    tokens = text.split()
    tokens = [w for w in tokens if w not in stop_words]
    tokens = [stemmer.stem(w) for w in tokens]
    return ' '.join(tokens)

# Kita akan membangun knowledge base berupa list dari (user_utterance_clean, bot_response)
# Caranya: Untuk setiap conversation, kita iterate turns.
# Setiap kali menemukan turn user, kita cek turn berikutnya apakah bot ada respons
# Jika ya, simpan (user_utterance_clean, bot_utterance) ke knowledge base

knowledge_base = []
for conv in data:
    turns = conv["turns"]
    for i in range(len(turns)-1):
        if turns[i]["speaker"] == "user" and turns[i+1]["speaker"] == "bot":
            user_utt = turns[i]["utterance"]
            bot_utt = turns[i+1]["utterance"]
            user_utt_clean = preprocess_text(user_utt)
            knowledge_base.append((user_utt_clean, bot_utt))

# Sekarang kita punya daftar pasangan (user_utterance_clean, bot_response)
# Buat DataFrame untuk memudahkan
df_kb = pd.DataFrame(knowledge_base, columns=["user_clean","bot_response"])

# Vectorize user_utterance_clean untuk similarity
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_kb["user_clean"])

def get_best_response(user_input):
    # Preprocess query
    user_query_clean = preprocess_text(user_input)
    q_vec = vectorizer.transform([user_query_clean])
    sims = cosine_similarity(q_vec, X)  # (1, n)
    best_idx = sims[0].argmax()
    best_score = sims[0][best_idx]
    # Threshold opsional, misal jika similarity < 0.2 balas default
    if best_score < 0.2:
        return "Maaf, saya belum memiliki informasi yang sesuai."
    else:
        return df_kb["bot_response"].iloc[best_idx]

# Contoh pengujian
test_input = "Kucing saya sering batuk dan hidungnya berair. Apa yang salah?"
response = get_best_response(test_input)
print("User:", test_input)
print("Chatbot:", response)

# Anda juga dapat menguji dengan input lain yang mirip dengan dataset
test_input2 = "Anjing saya tidak mau makan dan dia mencret"
response2 = get_best_response(test_input2)
print("User:", test_input2)
print("Chatbot:", response2)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


User: Kucing saya sering batuk dan hidungnya berair. Apa yang salah?
Chatbot: Gejala ini dapat menunjukkan Infeksi Saluran Pernapasan Atas. Apakah kucing Anda sering terpapar udara dingin atau debu?
User: Anjing saya tidak mau makan dan dia mencret
Chatbot: Jika tetap tidak mau makan selama lebih dari 24 jam, segera bawa anjing Anda ke dokter hewan untuk pemeriksaan lebih lanjut.


In [7]:
import ipywidgets as widgets
from IPython.display import display

# Kita asumsikan fungsi get_best_response dan variabel pendukung sudah didefinisikan sebelumnya:
# get_best_response(user_input) -> string

# Jika Anda ingin memasukkan konteks multi-turn, misalnya menggabungkan dua turn terakhir:
dialog_history = []
MAX_HISTORY = 2  # banyaknya turn sebelumnya yang akan digabung

def combined_context(new_input):
    # Menggabungkan turn terakhir di dialog_history dengan current input
    # untuk mempertahankan konteks multi-turn.
    history_context = dialog_history[-MAX_HISTORY:] if len(dialog_history) > MAX_HISTORY else dialog_history
    combined = " ".join(history_context + [new_input])
    return combined

# Input widget
input_box = widgets.Text(
    placeholder='Ketik pertanyaan Anda di sini...',
    description='User Input:',
    disabled=False
)

send_button = widgets.Button(
    description='Kirim',
    disabled=False,
    button_style='success'
)

output_area = widgets.Output()

def on_send_clicked(b):
    user_input = input_box.value.strip()
    if user_input:
        dialog_history.append(user_input)
        # Buat query dengan konteks multi-turn
        query_with_context = combined_context(user_input)
        response = get_best_response(query_with_context)
        with output_area:
            print(f"User: {user_input}")
            print(f"Chatbot: {response}\n")
    input_box.value = ""  # kosongkan setelah kirim

send_button.on_click(on_send_clicked)

display(input_box, send_button, output_area)

Text(value='', description='User Input:', placeholder='Ketik pertanyaan Anda di sini...')

Button(button_style='success', description='Kirim', style=ButtonStyle())

Output()